## Puppet Nodes

Another way to apply different rules to different systems is to use separate node definitions.**In Puppet terminology, a node is any system where we can run a Puppet agent**. It could be a physical workstation, a server, a virtual machine, or even a network router, as long as it has a Puppet agent and can apply the given rules.

When setting up Puppet, we usually have a **default node definition that lists the classes that should be included** for all the nodes.

```
node default {
    class {'sudo': }
    class { 'ntp':
        servers => ['ntp1.example.com', 'ntp2.example.com']
    }
}
```

What if you want some settings to only apply to some **specific nodes**? You can do that by adding more node definitions that list the classes that you want them to include, like this. We can see here that specific nodes in the fleet are identified by their FQDNs, or fully qualified domain names. 

```
node default {
    class {'sudo': }
    class { 'ntp':
        servers => ['ntp1.example.com', 'ntp2.example.com']
    }
    class { 'apache': }
}
```

We're listing the same classes because the classes included in the default node definition are only applied to the nodes that don't have an explicit entry. In other words, **when a node requests which rules it should apply, Puppet will look at the node definitions, figure out which one matches the node's FQDN, and then give only those rules.** 

Now, where's this information stored? The node definitions are typically stored in a file called site.pp, which isn't part of any module.

## Puppet's Certificate Infrastructure

in typical Puppet deployments, all managed machines and the fleet connect to a Puppet server. The client send their facts to the server, and the server then processes the manifests, generates the corresponding catalog, and sends it back to the clients who apply it locally.


 Puppet uses public key infrastructure, or PKI, to establish secure connections between the server and the clients. There's a bunch of different types of public key technologies. The one used by Puppet is secure sockets layer or SSL. This is the same technology used for encrypting transmissions over HTTPS. The clients use this infrastructure to check the server's identity, and the server uses it to check the client's identity, and all communication is done over an encrypted channel that uses these identities so it can't be intercepted by other parties. 
 
 
So how does this work? Each machine involved has a pair of keys related to each other, a private key and a public key.

The private key is secret, only known to that specific machine, the public key is shared with other machines involved. Machines can then use the standardized process to validate the identity of any other machine. 

The sender signs a message using the private key and the receiver validates the signature using the corresponding public key. Okay. 

But how do machines know which public keys to trust? This is where a certificate authority, or CA comes in. The **CA verifies the identity of the machine and then creates a certificate stating that the public key goes with that machine. After that, other machines can rely on that certificate to know that they can trust the public key, since it means the machine's identity has been verified. Puppet comes with its own certificate authority, which can be used to create certificates for each clients**

When a node checks into the Puppet master for the first time, it requests the certificate. The Puppet master looks at this request and if it can verify the nodes identity, it creates a certificate for that node. The system administrator can check the identity manually or use a process that does this automatically using additional information about the machines to verify their identity. When the agent node picks up this certificate, it knows it can trust the Puppet master, and the node can use the certificate from then on to identify itself when requesting a catalog. 

#### why need security?

1. First, Puppet rules can sometimes include confidential information that you don't want to fall in the wrong hand.
2. All sorts of things could go wrong if random computers start popping up in your network with the wrong settings.

## Setting up puppet Clients and Servers

**A Puppet deployment in action**. We've already installed the Puppet master package on this computer, so we'll use it as the **master**. Since this is a test deployment to demonstrate Puppet, we'll configure it to automatically sign the certificate requests of the nodes we add. But remember, if we were deploying this to real computers, we'd have to manually sign the requests or implement a proper validating script.

We'll do this by calling the Puppet command with the config parameter, and then saying that in this section master we want to set auto sign to true. All right. With that, **we can connect to the client that we want to manage using Puppet. We'll connect using SSH to a machine called web server. On this machine, we'll install the Puppet client which is shipped by the Puppet package.**

```
sudo puppet config --section master set autosign true

ssh webserver

sudo apt install puppet

sudo puppet config set server ubunut.example.com

sudo puppet agent -v --test
```

 Now we need to configure it to talk to the Puppet server that we're running on the other machine
 To do that, we'll use Puppet config like before but this time we'll tell it that we want to set the server to ubuntu.example.com.

Great. Now that we've configured the server, we can test the connection to the Puppet master by using the Puppet agent command passing dash v as before to get verbose output, and dash dash test to do a test run. 

 It first created an SSL key for the machine. It then read a bunch of information from the machine and used this to create a certificate request. The agent shows us the fingerprint of the certificate requested. If we were using manual signing, we could use this fingerprint to verify that the request and the server matches the one generated on the machine.
 
 But we see that this computer received a certificate and stored it locally. Once the certificate exchange completed, the agent retrieved all the information from the machine and sent it to the master. In exchange, it got back a catalog and applied it. The catalog applied almost immediately because we haven't actually configured any rules to be applied to our clients. 
 
We'll go back to our Puppet master and create a couple of node definitions. As we called out, node definitions are stored in a manifest file called site.pp, which is stored at the root of the nodes environment.

```
vim /setc/puppet/code/environments/production/manifests/site.pp


node weserver.example.com{
    class {'apache':}
}

node default {}
```

So the file that we need to create will be located in slash etc puppet code environments production manifests, and it will be called site.pp.


In this file, we'll create a couple of node definitions. We want to install Apache in our web server, so we'll create a node definition for the web server with the Apache class and node parameters for now.

Up to now, we've been doing manual runs of the Puppet agent for testing purposes. Now that we know it's working fine, we want to keep Puppet running automatically. That way, if we make changes to the configuration, clients will automatically apply those changes without us having to do any manual steps. So to do that, we'll **use the system CTL command, which lets us control the services that are enabled when the machine starts and those that are currently running. **

```
sudo puppet agent -v --test

sudo systemctl enable puppet
sudo systemctl start puppet
sudo systemctl status puppet
```

So we'll first tell the system CTL to enable the puppet service so that the agent gets started whenever the machine reboots, and then we'll tell system CTL to start the puppet service so that it starts running. Last step, we'll ask systems CTL for the status of the Puppet service to check that it's actually running.

## More Information about Deploying Puppet to Clients

Check out the following link for more information:

-http://www.masterzen.fr/2010/11/14/puppet-ssl-explained/